<a href="https://colab.research.google.com/github/RezuanChowdhuryRifat/Drug-Repurposing/blob/main/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing

In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib as plt
from itertools import combinations
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Create a code mapping dictionary

In [7]:
codes={}

## Drug's target

Load drug's target (from CTDbase)

In [ ]:
chemical_gene_df = pd.read_csv("/content/drive/MyDrive/Drug Repurposing/CTD_chem_gene_ixns.csv",index_col ="GeneID")

Load gene's (from CTDbase)

In [ ]:
gene_ids_df = pd.read_csv('/content/drive/MyDrive/Drug Repurposing/CTD_D000086382_genes_20240207180602.csv')
desired_gene_ids = gene_ids_df['Gene ID'].tolist()

In [ ]:
desired_rows = chemical_gene_df.loc[chemical_gene_df.index.isin(desired_gene_ids)]
output_filename = 'extracted_data.csv'

desired_rows.to_csv(output_filename)
extracted_df = desired_rows.sort_values(by='GeneID')
extracted_df = extracted_df[extracted_df['OrganismID'] == 9606.0]
extracted_df.to_csv('/content/drive/MyDrive/Drug Repurposing/extracted_file.csv')

In [3]:
gene_drug=pd.read_csv('/content/drive/MyDrive/Drug Repurposing/extracted_file.csv')

In [4]:
gene_drug.head()

,GeneID,ChemicalName,ChemicalID,CasRN,GeneSymbol,GeneForms,Organism,OrganismID,Interaction,InteractionActions,PubMedIDs
0,1,dorsomorphin,C516138,NaN,A1BG,mRNA,Homo sapiens,9606.0,[NOG protein co-treated with Phenylmercuric Ac...,affects^cotreatment|decreases^expression,27188386
1,1,Cyclosporine,D016572,59865-13-3,A1BG,mRNA,Homo sapiens,9606.0,Cyclosporine results in decreased expression o...,decreases^expression,25562108
2,1,Cyclosporine,D016572,59865-13-3,A1BG,mRNA,Homo sapiens,9606.0,Cyclosporine results in increased expression o...,increases^expression,27989131
3,1,Mercury,D008628,7439-97-6,A1BG,protein,Homo sapiens,9606.0,Mercury affects the expression of A1BG protein,affects^expression,22030286
4,1,Phenylmercuric Acetate,D010662,62-38-4,A1BG,mRNA,Homo sapiens,9606.0,Phenylmercuric Acetate results in decreased ex...,decreases^expression,26272509


Save the drug name and MeSH ID mapping for later use

In [8]:
codes['drugname2mesh']={row[0].upper():row[1] for idx, row in gene_drug[['ChemicalName','ChemicalID']].drop_duplicates().iterrows()}
codes['mesh2drugname']={row[0].upper():row[1] for idx, row in gene_drug[['ChemicalID','ChemicalName']].drop_duplicates().iterrows()}

In [9]:
codes['gene_symbol2id'] = {row[0].upper(): row[1] for _, row in gene_drug[['GeneSymbol', 'GeneID']].drop_duplicates().iterrows()}

In [11]:
gene_drug=gene_drug[['GeneID', 'ChemicalName']].drop_duplicates()
gene_drug['ChemicalName']=gene_drug['ChemicalName'].apply(lambda x: x.upper() if type(x)==str else x)

gene_drug['GeneID']=gene_drug['GeneID'].apply(lambda x: 'gene_'+str(x))
gene_drug['ChemicalName']=gene_drug['ChemicalName'].apply(lambda x: 'drug_'+x)

gene_drug.drop_duplicates(inplace=True)


## Pathways

Load pathways from CTDbase

In [12]:
pathways=pd.read_csv('/content/drive/MyDrive/Drug Repurposing/CTD_pathways.tsv',sep='\t')
path_sim=pd.concat([pd.DataFrame(list(combinations(pathway,2,)),columns=['gene1','gene2']) for pathway in pathways['Association inferred via'].apply(lambda x: x.split('|') if '|' in x else None).dropna().values]).drop_duplicates()

Load pathways from KEGG. The files were preprocessed as a pairwise gene

In [13]:
path_sim_kegg=pd.read_csv('/content/drive/MyDrive/Drug Repurposing/Kegg__pathways_prepocessed.txt', header=None,sep='\t')
path_sim_kegg.columns=['gene1', 'gene2', 'positive']
path_sim_kegg.replace('PP', 1, inplace=True)
path_sim_kegg.replace('PN', 0, inplace=True)

path_sim_kegg=path_sim_kegg.loc[path_sim_kegg['positive']==1, ['gene1','gene2']]

gene_name =pd.read_excel('/content/drive/MyDrive/Drug Repurposing/All_Human_Protein_Coding_Genes_3_27_2020.xlsx')
gene_dict= {row['Gene Id']:row['Gene Symbol'] for _, row in gene_name[['Gene Id','Gene Symbol']].iterrows()}

In [14]:
path_sim_kegg.dropna()

,gene1,gene2
0,10725,7124
1,10725,1437
2,4772,7124
3,4772,1437
4,4773,7124
...,...,...
29503,8733,84720
29504,8733,80055
29505,23556,55650
29506,2822,284098


In [15]:
path_sim_kegg['gene1']=path_sim_kegg['gene1'].apply(lambda x: gene_dict.get(x))
path_sim_kegg['gene2']=path_sim_kegg['gene2'].apply(lambda x: gene_dict.get(x))

In [16]:
path_sim_kegg.dropna(inplace=True)

Load PHARMAKB [pathways](https://www.pharmgkb.org/page/COVID)

In [17]:
pathway_ace_inhibitor=list(set(['ATP6AP2', 'MAPK1', 'AGTR2', 'ATP6AP2', 'REN', 'MAS1', 'TGFB1', 'MAPK3', 'ATP6AP2', 'MAPK3',
                       'AGTR1','TGFB1', 'MAPK1', 'NOS3', 'BDKRB2', 'BDKRB2', 'BDKRB1', 'NR3C2', 'CYP11B2', 'AGTR1', 'CYP11B2', 'AGTR1', 'AGT', 'KNG1', 'CYP11B2', 'ACE']))
pathway_fluv=['CYP1A2','CYP2C19','CYP3A']
pathway_losartan=list(set(['AGTR1','CYP2C9',"CYP3A4",'CYP2C9',"CYP3A4",'CYP2C9',"CYP3A4", 'CYP2C9',"CYP3A4", 'UGT1A1',"UGT2B7"]))

In [18]:
path_sim=pd.concat([path_sim]+[path_sim_kegg]+[pd.DataFrame(list(combinations(pathway,2,)),columns=['gene1','gene2']) for pathway in [pathway_ace_inhibitor,pathway_fluv,pathway_losartan]])

In [19]:
path_sim['gene1']=path_sim['gene1'].apply(lambda x: codes['gene_symbol2id'].get(x))
path_sim['gene2']=path_sim['gene2'].apply(lambda x: codes['gene_symbol2id'].get(x))
path_sim.dropna(inplace=True)
path_sim['gene1']=path_sim['gene1'].apply(lambda x: 'gene_'+str(int(x)))
path_sim['gene2']=path_sim['gene2'].apply(lambda x: 'gene_'+str(int(x)))
path_sim.drop_duplicates(inplace=True)

In [20]:
len(path_sim)

6239

## Phenotypes

Load phenotypes from CTDbase

In [24]:
phenotypes=pd.read_excel('/content/drive/MyDrive/Drug Repurposing/Phenotypes.xlsx')

In [25]:
phenotypes.head()

,Phenotype Term Name,Phenotype Term ID,Disease Name,Disease ID,Chemical Inference Network,Gene Inference Network,References
0,cell population proliferation,GO:0008283,COVID-19,MESH:D000086382,Azithromycin|Betulinic Acid|camostat|cepharant...,AGT|TYK2,197
1,apoptotic process,GO:0006915,COVID-19,MESH:D000086382,Acetazolamide|Azithromycin|Betulinic Acid|ceph...,BTK|IL1B|IL2RA|PLSCR1,347
2,viral life cycle,GO:0019058,COVID-19,MESH:D000086382,(2-tert-butoxy-1-(2-cyclohexyl-1-(1-formyl-2-(...,ACE2,73
3,cellular metabolic process,GO:0044237,COVID-19,MESH:D000086382,Acetazolamide|Azithromycin|Betulinic Acid|cann...,NaN,117
4,cell death,GO:0008219,COVID-19,MESH:D000086382,Betulinic Acid|Chloroquine|Dactinomycin|Iverme...,NaN,126


In [26]:
codes['phenotype_id_to_name']={row[0]:row[1] for idx, row in phenotypes[['Phenotype Term ID','Phenotype Term Name']].drop_duplicates().iterrows()}

Drugs and phenotypes

In [27]:
drug_phenotype=phenotypes['Chemical Inference Network'].dropna().apply(lambda x: x.split('|')).apply(pd.Series).merge(phenotypes['Phenotype Term ID'],left_index=True, right_index=True).melt(id_vars=['Phenotype Term ID'],value_name='drug').drop('variable', axis=1).dropna()
drug_phenotype['drug']=drug_phenotype['drug'].apply(lambda x: x.upper())
drug_phenotype.dropna(inplace=True)

drug_phenotype['Phenotype Term ID']=drug_phenotype['Phenotype Term ID'].apply(lambda x: 'phenotype_'+x)
drug_phenotype['drug']=drug_phenotype['drug'].apply(lambda x: 'drug_'+x)
drug_phenotype=drug_phenotype[['drug', 'Phenotype Term ID']]


Genes and phenotypes

In [28]:
gene_phenotype=phenotypes['Gene Inference Network'].dropna().apply(lambda x: x.split('|')).apply(pd.Series).merge(phenotypes['Phenotype Term ID'],left_index=True, right_index=True).melt(id_vars=['Phenotype Term ID'],value_name='gene').drop('variable', axis=1).dropna()
gene_phenotype['Phenotype Term ID']=gene_phenotype['Phenotype Term ID'].apply(lambda x: 'phenotype_'+x)

gene_phenotype['gene']=gene_phenotype['gene'].apply(lambda x: codes['gene_symbol2id'].get(x))
gene_phenotype.dropna(inplace=True)
gene_phenotype['gene']=gene_phenotype['gene'].apply(lambda x: 'gene_'+str(int(x)))
gene_phenotype=gene_phenotype[['gene', 'Phenotype Term ID']]

In [30]:
len(set(gene_drug['ChemicalName'].values).intersection(set(drug_phenotype['drug'].values)))

42

In [31]:
len(set(drug_phenotype['drug'].values))

51

## Baits and Prey

Load SARS-CoV-2 baits and host gene interaction from [Gorden et al. Nature 2020](https://www.nature.com/articles/s41586-020-2286-9#Sec36)

In [32]:
baits_prey=pd.read_excel('/content/drive/MyDrive/Drug Repurposing/SARS-CoV-2 and human protein interactions.xlsx')

In [33]:
baits_prey=baits_prey[['Bait', 'PreyGene']]

In [34]:
baits_prey['Bait'].nunique(), baits_prey['PreyGene'].nunique()

(27, 332)

In [35]:
baits_prey['Bait']=baits_prey['Bait'].apply(lambda x: 'bait_'+x)
baits_prey['PreyGene']=baits_prey['PreyGene'].apply(lambda x: codes['gene_symbol2id'].get(x))
baits_prey.dropna(inplace=True)
baits_prey['PreyGene']=baits_prey['PreyGene'].apply(lambda x: 'gene_'+str(int(x)))

In [36]:
#size of drug target, pathway, host gene, phenotype-related genes
gene_drug['GeneID'].nunique(),len(set(path_sim[['gene1','gene2']].values.ravel())),len(set(baits_prey['PreyGene'].unique())), gene_phenotype['gene'].nunique()

(9039, 1176, 247, 38)

In [37]:
#number of intersection between host genes and drug target
len(set(baits_prey['PreyGene'].unique()).intersection(gene_drug['GeneID'].unique()))

247

In [38]:
#intersection between target and pathways
len(set(gene_drug['GeneID'].unique()).intersection(set(path_sim[['gene1','gene2']].values.ravel())))

1176

In [39]:
#intersection between pathways and host gene
len(set(baits_prey['PreyGene'].unique()).intersection(set(path_sim[['gene1','gene2']].values.ravel())))

18

In [40]:
#intersection between pathways, target genes, host gene
len(set(baits_prey['PreyGene'].unique()).intersection(set(path_sim[['gene1','gene2']].values.ravel())).intersection(set(gene_drug['GeneID'].unique()) ))


18

## Item2idx mapping

Map all the entities (drugs, genes, phenotypes, baits) to ID

In [41]:
from sklearn.preprocessing import LabelEncoder

In [42]:
gene_drug.columns=['node1', 'node2']# gene, drug
path_sim.columns=['node1', 'node2']# gene1, gene2
baits_prey.columns=['node1','node2']#bait, preygene
gene_phenotype.columns=['node1', 'node2'] #gene, phenotype
drug_phenotype.columns=['node1','node2']#drug, phenotye

In [43]:
gene_drug['type']='gene-drug'
path_sim['type']='gene-gene'
baits_prey['type']='bait-gene'
gene_phenotype['type']='gene-phenotype'
drug_phenotype['type']='drug-phenotype'

edge_index=pd.concat([gene_drug, path_sim, baits_prey, gene_phenotype, drug_phenotype])

edge_index['node1']=edge_index['node1'].astype(str)
edge_index['node2']=edge_index['node2'].astype(str)

Label Encoders

In [44]:
le=LabelEncoder()
le.fit(np.concatenate((edge_index['node1'], edge_index['node2'])))

LabelEncoder()

In [45]:
edge_index['node1']=le.transform(edge_index['node1'])
edge_index['node2']=le.transform(edge_index['node2'])

In [46]:
len(le.classes_)

21306

## Import pre-trained embedding

Obtain pre-trained entity embedding from [DRKG](https://github.com/gnn4dr/DRKG)

In [47]:
import csv

In [48]:
!wget https://dgl-data.s3-us-west-2.amazonaws.com/dataset/DRKG/drkg.tar.gz

--2024-03-14 23:04:06--  https://dgl-data.s3-us-west-2.amazonaws.com/dataset/DRKG/drkg.tar.gz
Resolving dgl-data.s3-us-west-2.amazonaws.com (dgl-data.s3-us-west-2.amazonaws.com)... 52.92.180.202, 52.218.222.41, 52.92.162.50, ...
Connecting to dgl-data.s3-us-west-2.amazonaws.com (dgl-data.s3-us-west-2.amazonaws.com)|52.92.180.202|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 216650245 (207M) [application/x-tar]
Saving to: ‘drkg.tar.gz’

drkg.tar.gz         100%[===================>] 206.61M  29.6MB/s    in 6.9s    

2024-03-14 23:04:13 (30.0 MB/s) - ‘drkg.tar.gz’ saved [216650245/216650245]



In [ ]:
#Get pretrained embedding
entity_emb=np.load(data_path+'DRKG/embed/DRKG_TransE_l2_entity.npy')
emb_size=entity_emb.shape[1]

In [ ]:
entity_idmap_file = data_path+'DRKG/embed/entities.tsv'
relation_idmap_file = data_path+'DRKG/embed/relations.tsv'

In [ ]:
baits_drkg=['Disease::'+entity.split('_')[1] for entity  in le.classes_ if entity.split('_')[0]=='bait']
gene_drkg = ['Gene::'+entity.split('_')[1] for entity in le.classes_ if entity.split('_')[0]=='gene']
phenotype_drkg=['Biological Process::'+entity.split('_')[1] for entity in le.classes_ if entity.split('_')[0]=='phenotype']

Map the DRKG's DrugBank ID to MeSH ID

In [ ]:
drugname2external=pd.concat([pd.read_csv(data_path+'CTD/alldrugbank.csv', index_col=0).rename(columns={'drugbank_id':'id'}),
           pd.read_csv(data_path+'CTD/nondrugbank.csv', index_col=0).rename(columns={'chembl':'id'})]).groupby('drugname', as_index=False).first()
drugname2id={row[0].upper():row[1] for _, row in drugname2external[['drugname', 'id']].iterrows()}
drug_drkg = ['Compound::'+drugname2id.get(entity.split('_')[1],'') for entity in le.classes_ if entity.split('_')[0]=='drug']

Get drugname/disease name to entity ID mappings

In [ ]:

entity_map = {}
entity_id_map = {}
relation_map = {}
with open(entity_idmap_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['name','id'])
    for row_val in reader:
        entity_map[row_val['name']] = int(row_val['id'])
        entity_id_map[int(row_val['id'])] = row_val['name']

with open(relation_idmap_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['name','id'])
    for row_val in reader:
        relation_map[row_val['name']] = int(row_val['id'])

# handle the ID mapping
bait_ids = []
gene_ids = []
drug_ids = []
phenotype_ids = []


for bait in baits_drkg:
    bait_ids.append(entity_map.get(bait))

for gene in gene_drkg:
    gene_ids.append(entity_map.get(gene))

for drug in drug_drkg:
    drug_ids.append(entity_map.get(drug))

for phenotype in phenotype_drkg:
    phenotype_ids.append(entity_map.get(phenotype))


In [ ]:
bait_emb=np.array([entity_emb[bait_id] if bait_id is not None else np.zeros(emb_size) for bait_id in bait_ids ])
drug_emb=np.array([entity_emb[drug_id] if drug_id is not None else np.zeros(emb_size) for drug_id in drug_ids ])
gene_emb=np.array([entity_emb[gene_id] if gene_id is not None else np.zeros(emb_size) for gene_id in gene_ids ])
phenotype_emb=np.array([entity_emb[phenotype_id] if phenotype_id is not None else np.zeros(emb_size) for phenotype_id in phenotype_ids ])


In [ ]:
#How many missing in drugs?
len(drug_ids),len([gene_id for gene_id in drug_ids if gene_id is not None])

(3635, 1138)

In [ ]:
#How many missing in genes?
len(gene_ids),len([gene_id for gene_id in gene_ids if gene_id is not None])

(5677, 5666)

In [ ]:
#How many missing in phenotypes?
len(phenotype_ids),len([gene_id for gene_id in phenotype_ids if gene_id is not None])

(1285, 970)

The pre-trained embedding is now serived as a node feature

In [ ]:
node_features=np.concatenate((bait_emb, drug_emb, gene_emb, phenotype_emb))

## Save as pickle

In [ ]:
edge_index.to_pickle(data_path+'edge_index_'+exp_id+'.pkl')
pickle.dump(le, open(data_path+'LabelEncoder_'+exp_id+'.pkl','wb'))
pickle.dump(node_features, open(data_path+'node_feature_'+exp_id+'.pkl', 'wb'))
pickle.dump(codes, open(data_path+'codes_'+exp_id+'.pkl','wb'))